In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loandata/Loan payments data.csv


In [3]:
!conda install -y -c conda-forge pytorch-lightning 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pytorch-lightning


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    future-0.18.2              |   py37h89c1867_3         714 KB  conda-forge
    pytorch-lightning-1.2.3    |     pyhd8ed1ab_0         277 KB  conda-forge
    tensorboard-2.4.0          |     pyhc547734_0         8.8 MB
    tensorboard-plugin-wit-1.8.0|     pyh44b312d_0         670 KB  conda-forge
    werkzeug-1.0.1             |     pyh9f0ad1d_0         239 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        10.7 MB

The following NEW packages will be INSTALLED:

  absl-py            conda-forge/noarch::absl-py-0.12.0-pyhd8ed1ab_0
  future           

In [5]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import pytorch_lightning as pl
# from pytorch_lightning.logging import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [47]:
sample_df = pd.read_csv("/kaggle/input/loandata/Loan payments data.csv")

In [48]:
sample_df["loan_status"].unique()



array(['PAIDOFF', 'COLLECTION', 'COLLECTION_PAIDOFF'], dtype=object)

In [49]:
def to_categorical(y):
    """ 1-hot encodes a tensor """
    possibilities = {
        "PAIDOFF": 0,
        "COLLECTION": 1,
        "COLLECTION_PAIDOFF": 2
    }
    return possibilities[y]

In [50]:
class LoanDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, loan_file):
        temp_df = pd.read_csv(loan_file)
        temp_df
        self.tabular = pd.read_csv(loan_file)

    def __len__(self):
        return len(self.tabular)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tabular = self.tabular.iloc[idx, 0:]

        y = to_categorical(tabular["loan_status"])

        tabular = tabular[["Principal", "terms", "age"]]
        tabular = tabular.tolist()
        tabular = torch.FloatTensor(tabular)

        return tabular, y

In [54]:
class LitClassifier(pl.LightningModule):
    def __init__(
        self, lr: float = 1e-3, num_workers: int = 4, batch_size: int = 32,
    ):
        super().__init__()
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size

        self.ln4 = nn.Linear(3, 10)
        self.relu = nn.ReLU()
        self.ln5 = nn.Linear(10, 10)
        self.ln6 = nn.Linear(10, 5)
        self.ln7 = nn.Linear(5, 1)
        
    def forward(self, tab):
        tab = self.ln4(tab)
        tab = self.relu(tab)
        tab = self.ln5(tab)
        tab = self.relu(tab)
        tab = self.ln6(tab)
        tab = self.relu(tab)

        return self.ln7(tab)
    
    def training_step(self, batch, batch_idx):
        tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(tabular))
        y_pred = y_pred.double()

        loss = criterion(y_pred, y)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(tabular))
        y_pred = y_pred.double()

        val_loss = criterion(y_pred, y)

        return {"val_loss": val_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"val_loss": avg_loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        tabular, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(tabular))
        y_pred = y_pred.double()

        test_loss = criterion(y_pred, y)

        return {"test_loss": test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        logs = {"test_loss": avg_loss}
        return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}
    
    def setup(self, stage):

        loan_data = LoanDataset(loan_file="/kaggle/input/loandata/Loan payments data.csv")

        train_size = int(0.80 * len(loan_data))
        val_size = int((len(loan_data) - train_size) / 2)
        test_size = int((len(loan_data) - train_size) / 2)

        self.train_set, self.val_set, self.test_set = random_split(loan_data, (train_size, val_size, test_size))

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=(self.lr))

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)

In [55]:
model = LitClassifier()
trainer = pl.Trainer(gpus=0)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [56]:
trainer.fit(model)


  | Name | Type   | Params
--------------------------------
0 | ln4  | Linear | 40    
1 | relu | ReLU   | 0     
2 | ln5  | Linear | 110   
3 | ln6  | Linear | 55    
4 | ln7  | Linear | 6     
--------------------------------
211       Trainable params
0         Non-trainable params
211       Total params
0.001     Total estimated model params size (MB)


Validation sanity check: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The validation_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on

Training: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

...

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [57]:
trainer.test(model)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: |          | 0/? [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.39325258011619246}
--------------------------------------------------------------------------------


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The testing_epoch_end should not return anything as of 9.1. To log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


[{'test_loss': 0.39325258011619246}]